In [31]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext import data
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import argparse
import os
import datetime, time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence



In [32]:
# Mount Google drive to load data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [33]:
class CNN_LSTM_2_conv(nn.Module):
    def __init__(self, embedding_dim, vocab, hidden_dim=100):
        super(CNN_LSTM_2_conv, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(vocab.vectors)
        self.hidden_dim = hidden_dim

        self.conv1 = nn.Conv2d(1, 50,kernel_size=(2,embedding_dim)) #in_channels, out_chanels, kernel_size
        self.conv2 = nn.Conv2d(1, 50,kernel_size=(2,embedding_dim)) #in_channels, out_chanels, kernel_size
        
        
        self.lstm1 = nn.LSTM(embedding_dim,hidden_dim)
        target_size = 1
        self.fc1 = nn.Linear(hidden_dim, target_size)

    def forward(self, x, lengths):
    #   embeds = self.word_embeddings(sentence)
    #     lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
    #     tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
    #     tag_scores = F.log_softmax(tag_space, dim=1)
    #     return tag_scores
        
        x = self.embedding(x)
        # print('x.shape',x.shape)

        x = x.unsqueeze(0)
        # print('x = x.unsqueeze(0)',x.shape)
        x = x.transpose(1,2) # swaps 2nd and 3rd dimension
        # print('x = x.transpose(1,2)',x.shape)
        x = x.transpose(0,1) # swaps 1st and 2nd dimension, now it has the correct input (n_samples,channels, height, width)
        # print('x = x.transpose(0,1)',x.shape)
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x)) 
        # print('x1 = F.relu(self.conv1(x))',x1.shape)
        
        x1 = x1.squeeze(3)
        x2 = x2.squeeze(3)
        # print('x1 = x1.squeeze(3) hahaha',x1.shape)
        pool1 = nn.MaxPool1d(x1.size(2), 1)
        pool2 = nn.MaxPool1d(x2.size(2), 1) #maxpool
        
        x1 = pool1(x1)
        x2 = pool2(x2)
        # print('x1 = x1.squeeze(3) here',x1.shape)

        x = torch.cat((x1,x2),1)
        # print('pool1 = nn.MaxPool1d(x1.size(2), 1)',x.shape)
        # x = concat.view(-1,100)
        # print('x = x.',x.shape)
        
        x = x.transpose(1,2)
        # print('x === x.squeeze(1)',x.shape)
        x = x.transpose(0,1)
        # print('x === x.squeeze(0)',x.shape)


        #  pack the word embeddings in the batch together and 
        # run the RNN on this object
        # x = pack_padded_sequence(x,lengths) 
        # print('x.shape',x.shape)
        x,(h,c) = self.lstm1(x)
        # print('x.shape',x.shape)
        # print('h.shape',h.shape)
        # print('len(h)',len(h))
        # print('(x.view(len(lengths),-1)).shape',(x.view(len(lengths),-1)).shape)
        h_result = self.fc1(h.squeeze(0))
        # h_result = F.log_softmax(tag_space, dim=1) --> change to this with multiple cat
        h_result = torch.sigmoid(h_result)
        

        return h_result.squeeze(1)

In [34]:
def read_vocab(path):
    import pickle
    pkl_file = open(path, 'rb')
    vocab = pickle.load(pkl_file)
    pkl_file.close()
    return vocab


In [37]:
# data_path ='/content/drive/MyDrive/ece367/ECE324 project/dataProcessing/binary_data^6_aug_v2/identity_hate/test.csv'
data_path ='/content/drive/MyDrive/ece367/ECE324 project/dataProcessing/binary_data^6/obscene/test.csv'
test_data = pd.read_csv(data_path, sep=',')
print(test_data)

# model_path          ='/content/drive/MyDrive/ece367/ECE324 project/saved_model_vocab/Nov30 2:31pm AUG/'
model_path          ='/content/drive/MyDrive/ece367/ECE324 project/saved_model_vocab/Nov30 2:07pm/'

# toxic_model         =torch.load(model_path + 'toxic_aug_model_CNN_LSTM.pt')
# severe_toxic_model  =torch.load(model_path + 'severe_toxic_aug_model_CNN_LSTM.pt')
# obscene_model       =torch.load(model_path + 'obscene_aug_model_CNN_LSTM.pt')
# insult_model        =torch.load(model_path + 'insult_aug_model_CNN_LSTM.pt')
# threat_model        =torch.load(model_path + 'threat_aug_model_CNN_LSTM.pt')
# identity_hate_model =torch.load(model_path + 'identity_hate_aug_model_CNN_LSTM.pt')

obscene_model       =torch.load(model_path + 'obscene_model_CNN_LSTM_v3.pt')

# toxic_vocab = read_vocab(model_path + 'toxic_vocab (2).pkl')
# severe_toxic_vocab = read_vocab(model_path + 'severe_toxic_vocab (2).pkl')
obscene_vocab = read_vocab(model_path + 'obscene_vocab (1).pkl')
# insult_vocab = read_vocab(model_path + 'insult_vocab (2).pkl')
# threat_vocab = read_vocab(model_path + 'threat_vocab (2).pkl')
# identity_hate_vocab = read_vocab(model_path + 'identity_hate_vocab (2).pkl')

                    id  ... identity_hate
0     e8a64d43a4754f3c  ...             0
1     7d21325dd4b3466d  ...             0
2     fbc3024425805d29  ...             0
3     3f099a6d45912a39  ...             0
4     b40837a8ba4e5b7e  ...             0
...                ...  ...           ...
1685  49949dd2efdfd638  ...             0
1686  dd8e1086954e3e7c  ...             0
1687  9bbdc58507a2ddf4  ...             0
1688  25f678c101edbac6  ...             0
1689  437d4b831e70ed19  ...             0

[1690 rows x 8 columns]


In [38]:
def tokenizer(text):
    spacy_en = spacy.load("en")
    return [tok.text for tok in spacy_en(text)]

In [40]:
def predict(sentence):
    tokens = tokenizer(sentence)

    # token_ints = [toxic_vocab.stoi[tok] for tok in tokens]
    # token_tensor = torch.LongTensor(token_ints).view(-1,1)
    # lengths = torch.Tensor([len(token_ints)])
    # toxic_output            = toxic_model(token_tensor, lengths)

    # token_ints = [severe_toxic_vocab.stoi[tok] for tok in tokens]
    # token_tensor = torch.LongTensor(token_ints).view(-1,1)
    # lengths = torch.Tensor([len(token_ints)])
    # severe_toxic_output     = severe_toxic_model(token_tensor, lengths)

    token_ints = [obscene_vocab.stoi[tok] for tok in tokens]
    token_tensor = torch.LongTensor(token_ints).view(-1,1)
    lengths = torch.Tensor([len(token_ints)])
    obscene_output          = obscene_model(token_tensor, lengths)

    # token_ints = [insult_vocab.stoi[tok] for tok in tokens]
    # token_tensor = torch.LongTensor(token_ints).view(-1,1)
    # lengths = torch.Tensor([len(token_ints)])
    # insult_output           = insult_model(token_tensor, lengths)

    # token_ints = [threat_vocab.stoi[tok] for tok in tokens]
    # token_tensor = torch.LongTensor(token_ints).view(-1,1)
    # lengths = torch.Tensor([len(token_ints)])
    # threat_output           = threat_model(token_tensor, lengths)

    # token_ints = [identity_hate_vocab.stoi[tok] for tok in tokens]
    # token_tensor = torch.LongTensor(token_ints).view(-1,1)
    # lengths = torch.Tensor([len(token_ints)])
    # identity_hate_output    = identity_hate_model(token_tensor, lengths)

    # return toxic_output
    # return severe_toxic_output
    return obscene_output
    # return insult_output
    # return threat_output
    # return identity_hate_output

    # return [toxic_output, severe_toxic_output, obscene_output, insult_output, threat_output, identity_hate_output]

In [41]:
print(predict("i like cats and dogs but not you").item())

2.6909827283816412e-05


In [43]:
cm = [[0,0],[0,0]]
cols_target = "obscene"
# cols_target = ["toxic", "severe_toxic", "obscene", "insult", "threat", "identity_hate"]
for index, row in test_data.iterrows():
  print(cm)
  pred = predict(row["comment_text"]).item()
  #print(pred)
  if pred>0.5:
    pred = 1
  else:
    pred = 0

  if pred==1 and row[cols_target]==1:
    cm[0][0] += 1
  elif pred==0 and row[cols_target]==0:
    cm[1][1] += 1
  elif pred==1 and row[cols_target]==0:
    cm[0][1] += 1
  elif pred==0 and row[cols_target]==1:
    cm[1][0] += 1
  else:
    print(pred, row[cols_target])

print(cm)

[[0, 0], [0, 0]]
[[1, 0], [0, 0]]
[[1, 0], [0, 1]]
[[1, 0], [0, 2]]
[[2, 0], [0, 2]]
[[3, 0], [0, 2]]
[[3, 0], [0, 3]]
[[3, 0], [0, 4]]
[[3, 0], [0, 5]]
[[4, 0], [0, 5]]
[[5, 0], [0, 5]]
[[6, 0], [0, 5]]
[[7, 0], [0, 5]]
[[7, 0], [0, 6]]
[[8, 0], [0, 6]]
[[9, 0], [0, 6]]
[[10, 0], [0, 6]]
[[11, 0], [0, 6]]
[[12, 0], [0, 6]]
[[12, 0], [0, 7]]
[[12, 0], [0, 8]]
[[13, 0], [0, 8]]
[[13, 0], [0, 9]]
[[14, 0], [0, 9]]
[[14, 0], [0, 10]]
[[15, 0], [0, 10]]
[[16, 0], [0, 10]]
[[17, 0], [0, 10]]
[[18, 0], [0, 10]]
[[18, 0], [0, 11]]
[[18, 0], [0, 12]]
[[19, 0], [0, 12]]
[[20, 0], [0, 12]]
[[21, 0], [0, 12]]
[[21, 0], [0, 13]]
[[21, 0], [0, 14]]
[[22, 0], [0, 14]]
[[22, 0], [0, 15]]
[[22, 0], [0, 16]]
[[22, 0], [0, 17]]
[[23, 0], [0, 17]]
[[24, 0], [0, 17]]
[[24, 1], [0, 17]]
[[24, 1], [0, 18]]
[[25, 1], [0, 18]]
[[26, 1], [0, 18]]
[[26, 1], [0, 19]]
[[27, 1], [0, 19]]
[[27, 1], [0, 20]]
[[28, 1], [0, 20]]
[[28, 1], [0, 21]]
[[29, 1], [0, 21]]
[[29, 1], [0, 22]]
[[30, 1], [0, 22]]
[[30, 2], [0, 

In [ ]:
cm = [[[0,0],[0,0]],[[0,0],[0,0]],[[0,0],[0,0]],[[0,0],[0,0]],[[0,0],[0,0]],[[0,0],[0,0]]]
cols_target = ["toxic", "severe_toxic", "obscene", "insult", "threat", "identity_hate"]
for index, row in test_data.iterrows():
  print(cm)
  pred = predict(row["comment_text"])
  if len(pred)<10:
    pass
  #print(pred)
  for i in range(6):
    compare = 0
    if pred[i].item() >0.5:
      compare = 1
    if compare==1 and row[cols_target[i]]==1:
      cm[i][0][0] += 1
    elif compare==0 and row[cols_target[i]]==0:
      cm[i][1][1] += 1
    elif compare==1 and row[cols_target[i]]==0:
      cm[i][0][1] += 1
    else:
      cm[i][1][0] += 1

print(cm)

In [ ]:
pred = predict("i love cats and dogs but i hate this project")
print(pred[2].item())

0.8560025691986084
